In [1]:
# ==========================================
# PHASE 2: MQTT Logger (Final 6 Brokers)
# Goal: Connect to 6 Public Brokers (Including the new CN one)
# Language: English Only
# ==========================================

import paho.mqtt.client as mqtt
import pandas as pd
import time
import threading
import os
from datetime import datetime

# --- CONFIGURATION ---
DURATION_SECONDS = 1800  # 30 Minutes
OUTPUT_FILENAME = "mqtt_captured_data.csv"
INPUT_CSV = "custom_top_50_topics.csv"

# Updated Broker List (6 Brokers)
BROKERS = [
    {"address": "test.mosquitto.org", "port": 1883, "name": "Mosquitto"},
    {"address": "broker.hivemq.com", "port": 1883, "name": "HiveMQ"},
    {"address": "broker.emqx.io", "port": 1883, "name": "EMQX"},
    {"address": "broker.mqtt.cool", "port": 1883, "name": "MqttCool"},
    {"address": "public.mqttserver.eu", "port": 1883, "name": "MqttServerEU"},
    {"address": "broker-cn.emqx.io", "port": 1883, "name": "EmqxCN"} # New high-traffic broker
]

# --- LOAD TOPICS ---
def load_topics_from_csv():
    topics = []
    try:
        if not os.path.exists(INPUT_CSV):
            print(f"[ERROR] File '{INPUT_CSV}' not found!")
            return []
        
        df = pd.read_csv(INPUT_CSV)
        if 'Root Topic' in df.columns:
            raw_roots = df['Root Topic'].dropna().astype(str).tolist()
        else:
            raw_roots = df.iloc[:, 0].dropna().astype(str).tolist()
        
        for root in raw_roots:
            clean_root = root.strip().replace('#', '').strip('/')
            if clean_root:
                topics.append(f"{clean_root}/#")
        
        print(f"[SYSTEM] Loaded {len(topics)} topics from CSV.")
        return topics

    except Exception as e:
        print(f"[ERROR] Loading CSV failed: {e}")
        return []

TARGET_TOPICS = load_topics_from_csv()

if not TARGET_TOPICS:
    print("[STOP] No topics found.")
    raise SystemExit

# --- CLIENT SETUP ---
captured_data = []
data_lock = threading.Lock() 

def on_connect(client, userdata, flags, rc, properties=None):
    if rc == 0:
        print(f"🟢 [LINK] Connected to {userdata['name']}!", flush=True)
        for topic in TARGET_TOPICS:
            client.subscribe(topic, qos=0)
    else:
        print(f"🔴 [LINK] Connection Failed to {userdata['name']} (Code {rc})", flush=True)

def on_message(client, userdata, msg):
    try:
        payload_str = msg.payload.decode('utf-8', errors='ignore')
        p_type = "String"
        clean = payload_str.strip()
        if clean.startswith("{"): p_type = "JSON"
        elif clean.replace('.', '', 1).isdigit(): p_type = "Numeric"

        with data_lock:
            captured_data.append({
                "timestamp": datetime.now().strftime("%H:%M:%S"),
                "broker": userdata['name'],
                "topic": msg.topic,
                "payload_type": p_type,
                "qos": msg.qos,
                "retain": int(msg.retain)
            })
    except:
        pass 

# --- RUNNER ---
def start_logging():
    print(f"\n🚀 STARTING COLLECTION ({DURATION_SECONDS/60} Minutes)...", flush=True)
    print(f"📡 Brokers: {len(BROKERS)}", flush=True)
    
    clients = []
    for broker in BROKERS:
        try:
            client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, userdata={'name': broker['name']})
            client.on_connect = on_connect
            client.on_message = on_message
            client.connect(broker['address'], broker['port'], 60)
            client.loop_start()
            clients.append(client)
        except Exception as e:
            print(f"⚠️ Error connecting to {broker['name']}: {e}", flush=True)

    # Wait Loop
    start_time = time.time()
    try:
        print("\n[STATUS] Waiting for messages...", flush=True)
        while (time.time() - start_time) < DURATION_SECONDS:
            time.sleep(5) 
            elapsed = int(time.time() - start_time)
            count = len(captured_data)
            print(f"⏳ {elapsed}s elapsed... Captured: {count} messages", flush=True)

    except KeyboardInterrupt:
        print("\n🛑 Stopped by user.", flush=True)

    # Cleanup
    for c in clients:
        c.loop_stop()
        c.disconnect()

    # Save
    if captured_data:
        pd.DataFrame(captured_data).to_csv(OUTPUT_FILENAME, index=False)
        print(f"\n✅ DONE! Saved {len(captured_data)} messages to '{OUTPUT_FILENAME}'", flush=True)
    else:
        print("\n❌ No data captured.", flush=True)

if __name__ == "__main__":
    start_logging()

[SYSTEM] Loaded 50 topics from CSV.

🚀 STARTING COLLECTION (30.0 Minutes)...
📡 Brokers: 6
🟢 [LINK] Connected to Mosquitto!
🟢 [LINK] Connected to HiveMQ!
🟢 [LINK] Connected to EMQX!
🟢 [LINK] Connected to MqttCool!
🟢 [LINK] Connected to MqttServerEU!

[STATUS] Waiting for messages...
🟢 [LINK] Connected to EmqxCN!
⏳ 5s elapsed... Captured: 13872 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 10s elapsed... Captured: 14578 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 15s elapsed... Captured: 15134 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 20s elapsed... Captured: 15766 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 25s elapsed... Captured: 16377 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 30s elapsed... Captured: 16975 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 35s elapsed... Captured: 17554 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 40s elapsed... Captured: 18162 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 45s elapsed... Captured: 18863 messages
🟢 [LINK] Connected to EmqxCN!
⏳ 50s elapsed... Captured: 19467 message